In [1]:
import sys
sys.path

['/Users/ASi132/IdeaProjects/personal/DataAISummit2024/dataaisummit/demo_product',
 '/Users/ASi132/.pyenv/versions/3.10.5/lib/python310.zip',
 '/Users/ASi132/.pyenv/versions/3.10.5/lib/python3.10',
 '/Users/ASi132/.pyenv/versions/3.10.5/lib/python3.10/lib-dynload',
 '',
 '/Users/ASi132/.pyenv/versions/3.10.5/envs/dataaisummit2024_3105/lib/python3.10/site-packages',
 '/Users/ASi132/IdeaProjects/personal/DataAISummit2024']

In [2]:
sys.prefix

'/Users/ASi132/.pyenv/versions/3.10.5/envs/dataaisummit2024_3105'

In [3]:
import os
from pyspark.sql.session import SparkSession

# Setting the spark expectations environment variable to local, so that the spark session can be set accordingly for local testing
os.environ["SPARKEXPECTATIONS_ENV"] = "local"

In [4]:
# Setting up Kafka locally to stream the spark-expectations stats to Kafka
os.system(f"sh ./docker_scripts/docker_kafka_start_script.sh")

container is running


0

In [5]:
# Setting up the spark session for delta lake
spark = SparkSession.builder\
    .config( # type: ignore
        "spark.jars",
        "./jars/spark-sql-kafka-0-10_2.12-3.0.0.jar,"
        "./jars/kafka-clients-3.0.0.jar,"
        "./jars/commons-pool2-2.8.0.jar,"
        "./jars/spark-token-provider-kafka-0-10_2.12-3.0.0.jar"
    )\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")\
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.warehouse.dir", "/tmp/hive/warehouse")\
    .config("spark.driver.extraJavaOptions", "-Dderby.system.home=/tmp/derby")\
    .config("spark.jars.ivy", "/tmp/ivy2")\
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")\
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/ASi132/.pyenv/versions/3.10.5/envs/dataaisummit2024_3105/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /tmp/ivy2/cache
The jars for the packages stored in: /tmp/ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de4eb883-9cdb-49dd-8a39-3f7585dadb90;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 85ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||  

In [6]:
# cleanup the previous runs data if any existing        
spark.sql("drop database if exists  dq_spark_local cascade")
os.system("rm -rf /tmp/hive/warehouse/dq_spark_local.db")

0

In [7]:
# Creating the database for the spark expectations local testing
spark.sql("create database if not exists spark_catalog.dq_spark_local")
spark.sql(" use spark_catalog.dq_spark_local")

DataFrame[]

In [8]:
# Validate to expect zero tables in the database
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [9]:
# Setup the rules table schema
RULES_TABLE_SCHEMA = """ ( product_id STRING,
    table_name STRING,
    rule_type STRING,
    rule STRING,
    column_name STRING,
    expectation STRING,
    action_if_failed STRING,
    tag STRING,
    description STRING,
    enable_for_source_dq_validation BOOLEAN, 
    enable_for_target_dq_validation BOOLEAN,
    is_active BOOLEAN,
    enable_error_drop_alert BOOLEAN,
    error_drop_threshold INT )
"""

# Setup the rules
RULES_DATA = """ 
    ("your_product", "dq_spark_local.customer_order",  "row_dq", "customer_id_is_not_null", "customer_id", "customer_id is not null","drop", "validity", "customer_id should not be null", true, true,true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "row_dq", "sales_greater_than_two", "sales", "sales > 2", "drop", "accuracy", "sales value should be greater than two", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "row_dq", "discount_threshold", "discount", "discount*100 < 60","drop", "validity", "discount should be less than 40", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "row_dq", "ship_mode_in_set", "ship_mode", "lower(trim(ship_mode)) in('second class', 'standard class', 'standard class')", "drop", "validity", "ship_mode mode belongs in the sets", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "row_dq", "profit_threshold", "profit", "profit>0", "drop", "validity", "profit threshold should be greater than 0", true, true, true, true, 0)
    
    ,("your_product", "dq_spark_local.customer_order", "agg_dq", "sum_of_sales", "sales", "sum(sales)>10000", "ignore", "validity", "sum of sales should be greater than 10000",  true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "agg_dq", "sum_of_quantity", "quantity", "sum(quantity)>10000", "ignore", "validity", "sum of quantity should be greater than 10000", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "agg_dq", "distinct_of_ship_mode", "ship_mode", "count(distinct ship_mode)<=3", "ignore", "validity", "ship_mode's should not be more than 3", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "agg_dq", "row_count", "*", "count(*)>=10000", "ignore", "validity", "count should not be greater than 10000", true, true, true, false, 0)

    ,("your_product", "dq_spark_local.customer_order", "query_dq", "product_missing_count_threshold", "*", "((select count(distinct product_id) from product) - (select count(distinct product_id) from order))>(select count(distinct product_id) from product)*0.2", "ignore", "validity", "row count threshold", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "query_dq", "product_category", "*", "(select count(distinct category) from product) < 5", "ignore", "validity", "distinct product category", true, true, true, false, 0)
    ,("your_product", "dq_spark_local.customer_order", "query_dq", "row_count_in_order", "*", "(select count(*) from order)<10000", "ignore", "accuracy", "count of the row in order dataset", true, true, true, false, 0)
    
"""

In [10]:
# create rules table
spark.sql(f" CREATE TABLE dq_spark_local.dq_rules {RULES_TABLE_SCHEMA} USING DELTA")

# insert the rules data
spark.sql(f" INSERT INTO dq_spark_local.dq_rules  values {RULES_DATA} ")

24/01/03 18:49:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/03 18:49:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/01/03 18:49:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/01/03 18:49:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/01/03 18:49:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/01/03 18:49:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


DataFrame[]

In [11]:
# show rules
spark.sql("select * from dq_spark_local.dq_rules order by rule_type").show(truncate=False)

+------------+-----------------------------+---------+-------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+--------+--------------------------------------------+-------------------------------+-------------------------------+---------+-----------------------+--------------------+
|product_id  |table_name                   |rule_type|rule                           |column_name|expectation                                                                                                                                             |action_if_failed|tag     |description                                 |enable_for_source_dq_validation|enable_for_target_dq_validation|is_active|enable_error_drop_alert|error_drop_threshold|
+------------+-----------------------------+---------+-------------------------------+-----------+------------------

In [12]:
from pyspark.sql import DataFrame
from spark_expectations import _log
from spark_expectations.examples.base_setup import set_up_delta
from spark_expectations.core.expectations import (
    SparkExpectations,
    WrappedDataFrameWriter,
)
from spark_expectations.config.user_config import Constants as user_config

[2024-01-03 18:49:32,773] [INFO] [spark_expectations] {__init__.py:get_notifications_hook:40} - Loaded plugin with name: spark_expectations_email_notification and class: SparkExpectationsEmailPluginImpl
[2024-01-03 18:49:32,773] [INFO] [spark_expectations] {__init__.py:get_notifications_hook:40} - Loaded plugin with name: spark_expectations_slack_notification and class: SparkExpectationsSlackPluginImpl
[2024-01-03 18:49:32,773] [INFO] [spark_expectations] {__init__.py:get_notifications_hook:40} - Loaded plugin with name: spark_expectations_teams_notification and class: SparkExpectationsTeamsPluginImpl
[2024-01-03 18:49:32,776] [INFO] [spark_expectations] {__init__.py:get_sink_hook:28} - Loaded plugin with name: spark_expectations_kafka_write and class: SparkExpectationsKafkaWritePluginImpl


In [13]:
# Setup the writer configuration for the spark expectations
writer = WrappedDataFrameWriter().mode("append").format("delta")

# Setup the spark expectations object
se: SparkExpectations = SparkExpectations(
    product_id="your_product",
    rules_df=spark.table("dq_spark_local.dq_rules"),
    stats_table="dq_spark_local.dq_stats",
    stats_table_writer=writer,
    target_and_error_table_writer=writer,
    debugger=False,
    # stats_streaming_options={user_config.se_enable_streaming: False},
)

In [14]:
# setup the config for the spark expectations
user_conf = {
    user_config.se_notifications_enable_email: False,
    user_config.se_notifications_email_smtp_host: "mailhost.com",
    user_config.se_notifications_email_smtp_port: 25,
    user_config.se_notifications_email_from: "",
    user_config.se_notifications_email_to_other_mail_id: "",
    user_config.se_notifications_email_subject: "spark expectations - data quality - notifications",
    user_config.se_notifications_enable_slack: False,
    user_config.se_notifications_slack_webhook_url: "",
    user_config.se_notifications_on_start: True,
    user_config.se_notifications_on_completion: True,
    user_config.se_notifications_on_fail: True,
    user_config.se_notifications_on_error_drop_exceeds_threshold_breach: True,
    user_config.se_notifications_on_error_drop_threshold: 15,
}

In [15]:
# create product view
_df_product: DataFrame = (
        spark.read.option("header", "true")
        .option("inferSchema", "true")
        .csv("./resources/product.csv")
    )
_df_product.createOrReplaceTempView("product")

# create customer view
_df_customer: DataFrame = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("./resources/customer.csv")
)

_df_customer.createOrReplaceTempView("customer")

In [16]:
# create order view
_df_order: DataFrame = (
        spark.read.option("header", "true")
        .option("inferSchema", "true")
        .csv("./resources/order.csv")
    )
_df_order.createOrReplaceTempView("order")

In [17]:
# get the count of the order data
_df_order.count()

9987

In [18]:
# configure the spark expectations with the config and all the required parameters in the code
# Note the function should return a dataframe for the spark expectations to run or else it will throw an error
@se.with_expectations(
    target_table="dq_spark_local.customer_order",
    write_to_table=True,
    user_conf=user_conf,
    target_table_view="order",
)
def run_se() -> DataFrame:
    # _df_order: DataFrame = (
    #     spark.read.option("header", "true")
    #     .option("inferSchema", "true")
    #     .csv("./resources/order.csv")
    # )
    # _df_order.createOrReplaceTempView("order")

    return _df_order

In [19]:
# Running the function to build the dataframe and run the spark expectations
run_se()

[2024-01-03 18:49:34,627] [INFO] [spark_expectations] {expectations.py:wrapper:230} - The function dataframe is getting created
[2024-01-03 18:49:34,725] [INFO] [spark_expectations] {expectations.py:wrapper:277} - The function dataframe is created
[2024-01-03 18:49:34,726] [INFO] [spark_expectations] {expectations.py:wrapper:304} - started processing data quality rules for agg level expectations on soure dataframe
[2024-01-03 18:49:34,726] [INFO] [spark_expectations] {regulate_flow.py:func_process:92} - The data quality dataframe is getting created for expectations
[2024-01-03 18:49:34,896] [INFO] [spark_expectations] {regulate_flow.py:func_process:109} - The data quality dataframe is created for expectations
[2024-01-03 18:49:35,357] [INFO] [spark_expectations] {expectations.py:wrapper:329} - ended processing data quality rules for agg level expectations on source dataframe
[2024-01-03 18:49:35,357] [INFO] [spark_expectations] {expectations.py:wrapper:334} - started processing data qu

DataFrame[row_id: int, customer_id: string, product_id: string, order_id: string, order_date: string, ship_date: string, ship_mode: string, sales: string, quantity: string, discount: string, profit: double, meta_dq_run_id: string, meta_dq_run_date: string]

# Order of execution for the rules:

* Source Query DQ
* Source Agg DQ
* Row DQ
* Target Query DQ
* Target Agg DQ

In [20]:
# show tables
spark.sql("show tables").show()

+--------------+--------------------+-----------+
|     namespace|           tableName|isTemporary|
+--------------+--------------------+-----------+
|dq_spark_local|      customer_order|      false|
|dq_spark_local|customer_order_error|      false|
|dq_spark_local|            dq_rules|      false|
|dq_spark_local|            dq_stats|      false|
|              |            customer|      false|
|              |               order|      false|
|              |             product|      false|
+--------------+--------------------+-----------+



In [21]:
# review sample data in the stats table
spark.sql("select * from dq_spark_local.dq_stats").show(truncate=False)

+------------+-----------------------------+-----------+-----------+------------+-----------------+------------------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------

In [22]:
# review the schema of the stats table
spark.sql("select * from dq_spark_local.dq_stats").printSchema()

root
 |-- product_id: string (nullable = true)
 |-- table_name: string (nullable = true)
 |-- input_count: long (nullable = true)
 |-- error_count: long (nullable = true)
 |-- output_count: long (nullable = true)
 |-- output_percentage: float (nullable = true)
 |-- success_percentage: float (nullable = true)
 |-- error_percentage: float (nullable = true)
 |-- source_agg_dq_results: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- final_agg_dq_results: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- source_query_dq_results: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- final_query_dq_results: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |  

In [23]:
# get the count of the target table
spark.sql("select count(*) from dq_spark_local.customer_order").show()

+--------+
|count(1)|
+--------+
|    6118|
+--------+



In [24]:
# review the target table
spark.sql("select * from dq_spark_local.customer_order").show(truncate=False)

+------+-----------+---------------+--------------+----------+----------+--------------+--------+--------+--------+--------+-------------------------------------------------+-------------------+
|row_id|customer_id|product_id     |order_id      |order_date|ship_date |ship_mode     |sales   |quantity|discount|profit  |meta_dq_run_id                                   |meta_dq_run_date   |
+------+-----------+---------------+--------------+----------+----------+--------------+--------+--------+--------+--------+-------------------------------------------------+-------------------+
|1     |CG-12520   |FUR-BO-10001798|CA-2016-152156|11/8/2016 |11/11/2016|Second Class  |261.96  |2       |0       |41.9136 |your_product_1ff59ccc-aa9b-11ee-9109-babee37401fe|2024-01-04 06:49:32|
|2     |CG-12520   |FUR-CH-10000454|CA-2016-152156|11/8/2016 |11/11/2016|Second Class  |731.94  |3       |0       |219.582 |your_product_1ff59ccc-aa9b-11ee-9109-babee37401fe|2024-01-04 06:49:32|
|3     |DV-13045   |OFF-L

In [25]:
# get the count of the error table
spark.sql("select count(*) from dq_spark_local.customer_order_error").show()

+--------+
|count(1)|
+--------+
|    3869|
+--------+



In [26]:
spark.sql("select * from dq_spark_local.customer_order_error").show(truncate=False, n=1000)

+------+-----------+---------------+--------------+----------+----------+--------------+---------------------------------+--------------+--------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------+-------------------+
|row_id|customer_id|product_id     |order_id      |order_date|ship_date |ship_mode     |sales                            |quantity      |discount|profit    |meta_row_dq_results       

In [27]:
# reviews the stats from the kafka stream
spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "dq-sparkexpectations-stats")\
    .option("startingOffsets", "earliest")\
    .option("endingOffsets", "latest")\
    .load()\
    .selectExpr("cast(value as string) as stats_records")\
    .show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
# update the rules table to disable is_active flag for the rule - "sales_greater_than_two"
spark.sql("update dq_spark_local.dq_rules set is_active=false where rule='sales_greater_than_two'")

DataFrame[num_affected_rows: bigint]

In [29]:
# show the rules table  
spark.sql("select * from dq_spark_local.dq_rules order by rule_type, rule").show(truncate=False)

+------------+-----------------------------+---------+-------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+--------+--------------------------------------------+-------------------------------+-------------------------------+---------+-----------------------+--------------------+
|product_id  |table_name                   |rule_type|rule                           |column_name|expectation                                                                                                                                             |action_if_failed|tag     |description                                 |enable_for_source_dq_validation|enable_for_target_dq_validation|is_active|enable_error_drop_alert|error_drop_threshold|
+------------+-----------------------------+---------+-------------------------------+-----------+------------------

In [30]:
# Setup the writer configuration for the spark expectations
writer = WrappedDataFrameWriter().mode("append").format("delta")

# Setup the spark expectations object
se: SparkExpectations = SparkExpectations(
    product_id="your_product",
    rules_df=spark.table("dq_spark_local.dq_rules"),
    stats_table="dq_spark_local.dq_stats",
    stats_table_writer=writer,
    target_and_error_table_writer=writer,
    debugger=False,
    # stats_streaming_options={user_config.se_enable_streaming: False},
)

# Running for the second time to see the changes in the stats table
@se.with_expectations(
    target_table="dq_spark_local.customer_order",
    write_to_table=True,
    user_conf=user_conf,
    target_table_view="order",
)
def run_se() -> DataFrame:
    # _df_order: DataFrame = (
    #     spark.read.option("header", "true")
    #     .option("inferSchema", "true")
    #     .csv("./resources/order.csv")
    # )
    # _df_order.createOrReplaceTempView("order")

    return _df_order

run_se()

[2024-01-03 18:49:47,237] [INFO] [spark_expectations] {expectations.py:wrapper:230} - The function dataframe is getting created
[2024-01-03 18:49:47,318] [INFO] [spark_expectations] {expectations.py:wrapper:277} - The function dataframe is created
[2024-01-03 18:49:47,319] [INFO] [spark_expectations] {expectations.py:wrapper:304} - started processing data quality rules for agg level expectations on soure dataframe
[2024-01-03 18:49:47,319] [INFO] [spark_expectations] {regulate_flow.py:func_process:92} - The data quality dataframe is getting created for expectations
[2024-01-03 18:49:47,444] [INFO] [spark_expectations] {regulate_flow.py:func_process:109} - The data quality dataframe is created for expectations
[2024-01-03 18:49:47,661] [INFO] [spark_expectations] {expectations.py:wrapper:329} - ended processing data quality rules for agg level expectations on source dataframe
[2024-01-03 18:49:47,661] [INFO] [spark_expectations] {expectations.py:wrapper:334} - started processing data qu

DataFrame[row_id: int, customer_id: string, product_id: string, order_id: string, order_date: string, ship_date: string, ship_mode: string, sales: string, quantity: string, discount: string, profit: double, meta_dq_run_id: string, meta_dq_run_date: string]

In [31]:
# review sample data in the stats table
spark.sql("select * from dq_spark_local.dq_stats").show(truncate=False)

+------------+-----------------------------+-----------+-----------+------------+-----------------+------------------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------

In [32]:
should_not_fail_records = spark.sql("select * from order where sales<=2").count()
print(should_not_fail_records)

65


In [33]:
if should_not_fail_records == 3869-3804:
    print("Test Passed")

Test Passed


In [34]:
# get the count of the target table
spark.sql("select count(*) from dq_spark_local.customer_order").show()

+--------+
|count(1)|
+--------+
|   12301|
+--------+



In [35]:
# get the count of the error table
spark.sql("select count(*) from dq_spark_local.customer_order_error").show()

+--------+
|count(1)|
+--------+
|    7673|
+--------+



In [36]:
# reviews the stats from the kafka stream
spark.read.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option(
    "subscribe", "dq-sparkexpectations-stats"
).option("startingOffsets", "earliest").option(
    "endingOffsets", "latest"
).load().selectExpr(
    "cast(value as string) as stats_records"
).show(
    truncate=False
)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
# Failure Scenario
# Insert into dq_rules table to fail the job, have a dummy column to fail
spark.sql("insert into dq_spark_local.dq_rules values ('your_product', 'dq_spark_local.customer_order', 'row_dq', 'dummy', 'dummy', 'dummy', 'drop', 'dummy', 'dummy', true, true, true, false, 0)")

DataFrame[]

In [38]:
# Setup the writer configuration for the spark expectations
writer = WrappedDataFrameWriter().mode("append").format("delta")

# Setup the spark expectations object
se: SparkExpectations = SparkExpectations(
    product_id="your_product",
    rules_df=spark.table("dq_spark_local.dq_rules"),
    stats_table="dq_spark_local.dq_stats",
    stats_table_writer=writer,
    target_and_error_table_writer=writer,
    debugger=False,
    # stats_streaming_options={user_config.se_enable_streaming: False},
)


# Running for the second time to see the changes in the stats table
@se.with_expectations(
    target_table="dq_spark_local.customer_order",
    write_to_table=True,
    user_conf=user_conf,
    target_table_view="order",
)
def run_se() -> DataFrame:
    # _df_order: DataFrame = (
    #     spark.read.option("header", "true")
    #     .option("inferSchema", "true")
    #     .csv("./resources/order.csv")
    # )
    # _df_order.createOrReplaceTempView("order")

    return _df_order

try:
    run_se()
except Exception as e:
    print(e)

[2024-01-03 18:49:54,712] [INFO] [spark_expectations] {expectations.py:wrapper:230} - The function dataframe is getting created
[2024-01-03 18:49:54,737] [INFO] [spark_expectations] {expectations.py:wrapper:277} - The function dataframe is created
[2024-01-03 18:49:54,738] [INFO] [spark_expectations] {expectations.py:wrapper:304} - started processing data quality rules for agg level expectations on soure dataframe
[2024-01-03 18:49:54,738] [INFO] [spark_expectations] {regulate_flow.py:func_process:92} - The data quality dataframe is getting created for expectations
[2024-01-03 18:49:54,836] [INFO] [spark_expectations] {regulate_flow.py:func_process:109} - The data quality dataframe is created for expectations
[2024-01-03 18:49:55,030] [INFO] [spark_expectations] {expectations.py:wrapper:329} - ended processing data quality rules for agg level expectations on source dataframe
[2024-01-03 18:49:55,031] [INFO] [spark_expectations] {expectations.py:wrapper:334} - started processing data qu

In [39]:
# Exploring the stats table
# Total Input, Error and Output records
spark.sql("""
    SELECT SUM(input_count) AS total_input, 
    SUM(error_count) AS total_errors, 
    SUM(output_count) AS total_output 
    FROM  dq_spark_local.dq_stats
    """
).show()


+-----------+------------+------------+
|total_input|total_errors|total_output|
+-----------+------------+------------+
|      29961|        7673|       12301|
+-----------+------------+------------+



In [40]:
spark.sql("""
    SELECT product_id, round(avg(success_percentage), 2) AS avg_success, 
    round(avg(error_percentage), 2) AS avg_error 
    FROM dq_spark_local.dq_stats group by product_id
""").show()

+------------+-----------+---------+
|  product_id|avg_success|avg_error|
+------------+-----------+---------+
|your_product|      74.39|    25.61|
+------------+-----------+---------+



In [41]:
spark.sql(
    """
    SELECT product_id, key AS dq_status_type, value AS dq_status, COUNT(*) AS count
FROM dq_spark_local.dq_stats 
LATERAL VIEW explode(dq_status) AS key, value
GROUP BY product_id, key, value
order by product_id, key, value;
"""
).show()

+------------+---------------+---------+-----+
|  product_id| dq_status_type|dq_status|count|
+------------+---------------+---------+-----+
|your_product|   final_agg_dq|   Passed|    2|
|your_product|   final_agg_dq|  Skipped|    1|
|your_product| final_query_dq|   Passed|    2|
|your_product| final_query_dq|  Skipped|    1|
|your_product|         row_dq|   Failed|    1|
|your_product|         row_dq|   Passed|    2|
|your_product|     run_status|   Failed|    1|
|your_product|     run_status|   Passed|    2|
|your_product|  source_agg_dq|   Passed|    3|
|your_product|source_query_dq|   Passed|    3|
+------------+---------------+---------+-----+



In [42]:
spark.sql("""
    SELECT product_id, table_name, rule.rule_type, rule.description, rule.rule, rule.failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(row_dq_res_summary) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
""").show(truncate=False)

+------------+-----------------------------+---------+-----------------------------------------+----------------------+----------------+
|product_id  |table_name                   |rule_type|description                              |rule                  |failed_row_count|
+------------+-----------------------------+---------+-----------------------------------------+----------------------+----------------+
|your_product|dq_spark_local.customer_order|row_dq   |profit threshold should be greater than 0|profit_threshold      |2000            |
|your_product|dq_spark_local.customer_order|row_dq   |discount should be less than 40          |discount_threshold    |1108            |
|your_product|dq_spark_local.customer_order|row_dq   |sales value should be greater than two   |sales_greater_than_two|499             |
|your_product|dq_spark_local.customer_order|row_dq   |ship_mode mode belongs in the sets       |ship_mode_in_set      |2078            |
|your_product|dq_spark_local.customer_ord

In [43]:
spark.sql("""
    SELECT product_id, table_name, 'source_agg_dq' AS dq_type, rule.rule AS rule_type, rule.description, CAST(rule.failed_row_count AS INT) AS failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(source_agg_dq_results) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
    UNION ALL
    SELECT product_id, table_name, 'final_agg_dq' AS dq_type, rule.rule AS rule_type, rule.description, CAST(rule.failed_row_count AS INT) AS failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(final_agg_dq_results) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
    UNION ALL
    SELECT product_id, table_name, 'source_query_dq' AS dq_type, rule.rule AS rule_type, rule.description, CAST(rule.failed_row_count AS INT) AS failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(source_query_dq_results) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
    UNION ALL
    SELECT product_id, table_name, 'final_query_dq' AS dq_type, rule.rule AS rule_type, rule.description, CAST(rule.failed_row_count AS INT) AS failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(final_query_dq_results) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
    UNION ALL
    SELECT product_id, table_name, 'row_dq' AS dq_type, rule.rule AS rule_type, rule.description, CAST(rule.failed_row_count AS INT) AS failed_row_count
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(row_dq_res_summary) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0
""").show(truncate=False, n=1000)

+------------+-----------------------------+-------+----------------------+-----------------------------------------+----------------+
|product_id  |table_name                   |dq_type|rule_type             |description                              |failed_row_count|
+------------+-----------------------------+-------+----------------------+-----------------------------------------+----------------+
|your_product|dq_spark_local.customer_order|row_dq |profit_threshold      |profit threshold should be greater than 0|2000            |
|your_product|dq_spark_local.customer_order|row_dq |discount_threshold    |discount should be less than 40          |1108            |
|your_product|dq_spark_local.customer_order|row_dq |sales_greater_than_two|sales value should be greater than two   |499             |
|your_product|dq_spark_local.customer_order|row_dq |ship_mode_in_set      |ship_mode mode belongs in the sets       |2078            |
|your_product|dq_spark_local.customer_order|row_dq |pro

In [44]:
spark.sql("""
    select *, count(*) as failed_count from (
    SELECT product_id, table_name, 'source_agg_dq' AS dq_type, rule.rule AS rule_type, rule.description
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(source_agg_dq_results) exploded_table AS rule
    UNION ALL
    SELECT product_id, table_name, 'final_agg_dq' AS dq_type, rule.rule AS rule_type, rule.description
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(final_agg_dq_results) exploded_table AS rule
    UNION ALL
    SELECT product_id, table_name, 'source_query_dq' AS dq_type, rule.rule AS rule_type, rule.description
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(source_query_dq_results) exploded_table AS rule
    UNION ALL
    SELECT product_id, table_name, 'final_query_dq' AS dq_type, rule.rule AS rule_type, rule.description
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(final_query_dq_results) exploded_table AS rule
    UNION ALL
    SELECT product_id, table_name, 'row_dq' AS dq_type, rule.rule AS rule_type, rule.description
    FROM dq_spark_local.dq_stats 
    LATERAL VIEW explode(row_dq_res_summary) exploded_table AS rule
    WHERE CAST(rule.failed_row_count AS INT) > 0) group by all
""").show(truncate=False)

+------------+-----------------------------+---------------+-------------------------------+-----------------------------------------+------------+
|product_id  |table_name                   |dq_type        |rule_type                      |description                              |failed_count|
+------------+-----------------------------+---------------+-------------------------------+-----------------------------------------+------------+
|your_product|dq_spark_local.customer_order|source_agg_dq  |distinct_of_ship_mode          |ship_mode's should not be more than 3    |3           |
|your_product|dq_spark_local.customer_order|source_agg_dq  |row_count                      |count should not be greater than 10000   |3           |
|your_product|dq_spark_local.customer_order|final_agg_dq   |row_count                      |count should not be greater than 10000   |2           |
|your_product|dq_spark_local.customer_order|source_query_dq|product_missing_count_threshold|row count threshold 

In [45]:
# remove spark session
# spark.stop()

In [46]:
# remove the docker container
# os.system(f"sh ./docker_scripts/docker_kafka_stop_script.sh")